## I. Importing data

In [19]:
import pandas as pd
data_x = pd.read_csv('traininginputs.csv')
data_y = pd.read_csv('trainingoutput.csv')

## II. Cleaning data 

In [20]:
data = pd.merge(data_x, data_y, on='PROC_TRACEINFO', how ='inner')

Nous explorant notre base de donnée afin de se faire une idée sur son contenu. Pour cela commencons par voir les dimensions de notre dataset (nombre d'observations et de variables):

In [21]:
nbr_lignes, nbr_colonnes = data.shape
print("Nbr de lignes :", nbr_lignes )
print("Nbr de colonnes :",nbr_colonnes )

Nbr de lignes : 34515
Nbr de colonnes : 15


Nous avons donc 14 variables pour 34.515 individus. 

Vérifions s'il y a des valeurs manquantes.

In [22]:
data.isna().sum()

PROC_TRACEINFO                         0
OP070_V_1_angle_value                  0
OP090_SnapRingPeakForce_value          0
OP070_V_2_angle_value                  0
OP120_Rodage_I_mesure_value            0
OP090_SnapRingFinalStroke_value        0
OP110_Vissage_M8_torque_value          0
OP100_Capuchon_insertion_mesure    18627
OP120_Rodage_U_mesure_value            0
OP070_V_1_torque_value                 0
OP090_StartLinePeakForce_value         0
OP110_Vissage_M8_angle_value           0
OP090_SnapRingMidPointForce_val        0
OP070_V_2_torque_value                 0
Binar OP130_Resultat_Global_v          0
dtype: int64

Il semblerait que la variable 'OP100_Capuchon_insertion_mesure' compte 18.627 valeurs manquantes soit pour 53% des observations. Dans ce cas il serait difficile et pas pertinant de remplacer ces valeurs manquantes. Deux options s'offrirons a nous : utiliser des modèles qui gèrent les valeurs manquantes ou tout simplement supprimer cette variable. Les deux options seront explorées.

In [23]:
type_col = data.dtypes
print(type_col)

PROC_TRACEINFO                      object
OP070_V_1_angle_value              float64
OP090_SnapRingPeakForce_value      float64
OP070_V_2_angle_value              float64
OP120_Rodage_I_mesure_value        float64
OP090_SnapRingFinalStroke_value    float64
OP110_Vissage_M8_torque_value      float64
OP100_Capuchon_insertion_mesure    float64
OP120_Rodage_U_mesure_value        float64
OP070_V_1_torque_value             float64
OP090_StartLinePeakForce_value     float64
OP110_Vissage_M8_angle_value       float64
OP090_SnapRingMidPointForce_val    float64
OP070_V_2_torque_value             float64
Binar OP130_Resultat_Global_v        int64
dtype: object


La variable PROC_TRACEINFO est sous format 'object' et n a aucun interet. L énnoncé confirme qu il s agit Id de pieces. Nous décidons donc de la supprimer:

In [24]:
data = data.drop('PROC_TRACEINFO', axis =1 )

OK toutes les variables sont quantitatives. 
Par contre notre prédicteur (Binar OP130_Resultat_Global_v) devrait etre une variable catégorielle a deux modalités et non une variable continue. Nous procédons donc au changement de son type.

In [25]:
data['Binar OP130_Resultat_Global_v'] = data['Binar OP130_Resultat_Global_v'].astype('category')
print(data['Binar OP130_Resultat_Global_v'].dtype)

category


## III. Analyse des données 

L'analyse de donnée est un travail déterminant pour la bonne compréhension de nos données et permet parfois de mieux aiguiller le travail de modélisation. L'analyse de données permet d'étudier aussi bien la relation entre les différentes variables mais aussi de se faire une idée sur la population (exemple son homogénéité). Dans le cas ou l'on a une population hétérogène, il serait pertinant de créer plusieurs groupes d'observations, dans ce cas deux solutions sont envisageables : développer un modèle par groupe homoène d'invidus , ou encore rajouter une variable dans notre base de donnée qui porterait le groupe auquel appartiendrait chaque observation. L'inconvénient de cette stratégie est que pour chaque nouvelle observation , il faudra tout d'abord déterminer le groupe auquel elle appartient avant de l'intégrer dans le modèle.

In [26]:
print(data.describe())

       OP070_V_1_angle_value  OP090_SnapRingPeakForce_value  \
count           34515.000000                   34515.000000   
mean              159.906922                     156.915055   
std                15.662650                      11.271492   
min               101.800000                       0.000000   
25%               148.700000                     149.210000   
50%               158.000000                     156.180000   
75%               169.300000                     164.380000   
max               198.300000                     196.920000   

       OP070_V_2_angle_value  OP120_Rodage_I_mesure_value  \
count           34515.000000                 34515.000000   
mean              159.618236                   113.350222   
std                15.091490                     3.528522   
min                82.000000                    99.990000   
25%               149.400000                   111.040000   
50%               158.700000                   113.160000   
75%  

Contenu de la présence de valeurs manquantes dans l une de nos varibles nous décidons de partir un XGboost directement en premier essai car il intégre la possibilité de gestion de valeurs manquantes.

# IV. Modélisation

In [27]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import make_scorer, roc_auc_score

Séparer les données en prédicteur (y) / variables prédictives (X) :

In [28]:
y = data['Binar OP130_Resultat_Global_v']
X = data.drop('Binar OP130_Resultat_Global_v', axis =1)

Nous séparons notre dataset en train/test en choisissant d'allouer 20% aux données test.  

In [29]:
X_train, X_test, y_train, y_test= train_test_split(X, y,test_size=0.2, random_state=123)

### Option 1 : modélisation avec XG Boost:

XGBoost, en tant qu'algorithme de boosting basé sur des arbres de décision, n'a  pas besoin de centrer ou de réduire les données en amont. Nous procédons donc directement a la modélisation. 

Par défaut, XGBoost est capable de gérer les valeurs manquantes automatiquement.

#### 1.1 Recherche de paramètres avec la méthode dite 'brute force': 

In [40]:
# définition du modèle xgboost

model = xgb.XGBClassifier()

# Spécifiez la grille des hyperparamètres à explorer
param_grid = {
    'learning_rate': [0.055, 0.066 ,0.06, 0.2, 0.3],
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 4, 5,6],
    'min_child_weight': [1, 2, 3],
    'gamma': [0.08, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 0.7],
}

# Créez un objet GridSearchCV pour la recherche par grille
grid_search = GridSearchCV(model, param_grid, scoring='roc_auc', cv=5, verbose=1, n_jobs=-1)

# Entraînez le modèle en utilisant la recherche par grille
grid_search.fit(X_train, y_train)

# Affichez les meilleurs hyperparamètres trouvés
print("Meilleurs hyperparamètres:", grid_search.best_params_)


Fitting 5 folds for each of 3645 candidates, totalling 18225 fits
Meilleurs hyperparamètres: {'colsample_bytree': 0.8, 'gamma': 0.1, 'learning_rate': 0.06, 'max_depth': 4, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 1.0}


In [41]:
best_params = grid_search.best_params_ #enregistre les meilleurs paramètres trouvés.

best_model = xgb.XGBClassifier(**best_params)  # Remplacez XGBClassifier par le modèle que vous utilisez

best_model.fit(X_train, y_train) #entrainer le modèle avec les meilleurs paramètres trouvés.

y_pred_proba = best_model.predict_proba(X_test)[:, 1]  # prédire sur la partie X_test par vos données de test

roc_auc = roc_auc_score(y_test, y_pred_proba)  # Remplacez y_test par vos étiquettes de test

print("ROC AUC Score:", roc_auc)



ROC AUC Score: 0.7066849126907246


Ok 0.61
Ok 0.6955
ROC AUC Score: 0.7066849126907246 -> Meilleurs hyperparamètres: {'colsample_bytree': 0.8, 'gamma': 0.1, 'learning_rate': 0.06, 'max_depth': 4, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 1.0}
